In [ ]:
import os
os.chdir(r'D:\Work\VGG19')

In [ ]:
from keras.applications.vgg19 import VGG19
import keras

In [ ]:
from keras.preprocessing import image

In [ ]:
train_dir = 'train'
test_dir = 'test'
# 批次处理大小
batch_size = 100

In [ ]:
# 图像像元值归一化处理器 0-255 ~ 0-1
train_datagen = image.ImageDataGenerator(rescale=1/255)
test_datagen = image.ImageDataGenerator(rescale=1/255)

In [ ]:
# 影像处理，分类器
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),      # resize大小
    batch_size=batch_size,
    class_mode='categorical')         

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical')

In [ ]:
base_model = VGG19(weights='imagenet', 
                   include_top=False,
                   input_shape=(224, 224, 3))

In [ ]:
base_model.summary()

In [ ]:
from keras.layers import Dense, GlobalMaxPooling2D, Dropout
import numpy as np
np.set_printoptions(suppress = True)

In [ ]:
# 使用VGG 19提取图片特征
def extractFeatures(data_generator):
    i = 0
    sample_count = len(data_generator.filenames)
    features = []
    labels = []
    for inputs_batch, labels_batch in data_generator:
        features_batch = base_model.predict(inputs_batch)
        features.append(features_batch)
        labels.append(labels_batch)
        i += 1
        if i*batch_size >= sample_count:
            break
    features = np.concatenate(features)
    labels = np.concatenate(labels)
    return features, labels 

In [ ]:
train_features, train_lables = extractFeatures(train_generator)

In [ ]:
test_features, test_lables = extractFeatures(test_generator)

In [ ]:
model = keras.Sequential()
# 池化输入层，压缩数据
model.add(GlobalMaxPooling2D(input_shape=(7,7,512)))
# 全连接层，特征提取器
model.add(Dense(512, activation='relu'))
# Dropout 防止过拟合 
model.add(Dropout(0.5))
# 输出层 sigmoid激活函数用于二分类输出 softmax用于多分类输出
model.add(Dense(2, activation='sigmoid'))

In [ ]:
# 模型编译
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# 模型训练
history = model.fit(
    train_features,
    train_lables,
    epochs=20,
    batch_size=batch_size,
    validation_data=(test_features, test_lables)
)

In [ ]:
model.evaluate(test_features, test_lables)

In [ ]:
from glob import glob

In [ ]:
files = glob("predict/*/*.jpg")

In [ ]:
files

In [ ]:
predict_data = []
for file in files:
    img = image.image_utils.load_img(file, target_size=(224, 224))
    data = image.image_utils.img_to_array(img) / 255
    predict_data.append(data)
predict_data = np.array(predict_data)

In [ ]:
predict_data.shape

In [ ]:
predict_features = base_model.predict(predict_data)

In [ ]:
predict_features.shape

In [ ]:
predict_result = model.predict(predict_features)

In [ ]:
a = np.around(predict_result, decimals=0)

In [ ]:
import matplotlib.pyplot as plt

In [1]:
import tensorflow as tf


In [4]:

print(tf.__version__)

2.12.0
